In [1]:
%matplotlib inline
from taxi_pakage import *
taxi = pd.read_csv("edited_train.csv")

In [2]:
# 날씨 데이터 생성
weather_event = ['20160110', '20160113', '20160117', '20160123', '20160205', '20160208', '20160215', '20160216',
                 '20160224', '20160225', '20160314', '20160315', '20160328', '20160329', '20160403', '20160404',
                 '20160530', '20160628']

weather_event = pd.Series(pd.to_datetime(weather_event, format = '%Y%m%d')).dt.date
weather_event = weather_event.astype('<U32')
weather_event = list(weather_event)

taxi["y-m-d"] = pd.to_datetime(taxi["pickup_datetime"]).apply(lambda x: x.strftime("%Y-%m-%d"))
taxi["extreme_weather"] = taxi["y-m-d"].apply(lambda x: 1 if x in weather_event else 0)
taxi["weather_event"] = taxi["extreme_weather"] # 날씨 (1:자연재해,  0:자연재해X)
taxi.drop(['y-m-d', 'extreme_weather'], axis=1, inplace=True)

In [3]:
taxi['sqrt_log_dist'] = taxi['dist'].apply(lambda x: np.sqrt(np.log1p(x)))
taxi['log_duration'] = taxi['trip_duration'].apply(lambda x: np.log1p(x))

In [4]:
taxi['velo'] = taxi['dist']/taxi['trip_duration']*3600 # 시속
taxi['no_passenger'] = taxi['passenger_count'].apply(lambda x: 1 if x == 0 else 0)

In [11]:
# 아웃라이어 제거
taxi = taxi[taxi['trip_duration'] < 1500000].reset_index(drop=True)
taxi = taxi[taxi['velo']<100]

In [12]:
filtered = taxi[taxi['velo']>2]

In [8]:
# origin data model
model = sm.OLS.from_formula("log_duration ~ scale(hour) +scale(hour**2) +scale(hour**3) + scale(hour**4) +scale(hour**5) +scale(hour**6)+scale(hour**7) + scale(hour**8) + scale(hour**9)", data = taxi)
result2 = model.fit_regularized(alpha=0.01, L1_wt=1)
print(result2.params)

Intercept           6.457340
scale(hour)         0.037837
scale(hour ** 2)    0.000000
scale(hour ** 3)    0.000000
scale(hour ** 4)    0.000000
scale(hour ** 5)    0.000000
scale(hour ** 6)   -0.013010
scale(hour ** 7)   -0.008714
scale(hour ** 8)    0.000000
scale(hour ** 9)    0.000000
dtype: float64


In [18]:
score, result_set = cross_validater("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*C(work)\
                             + scale(sqrt_log_dist)*scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(hour)+ scale(hour**6)+ scale(hour**7)\
                             + scale(month)+ scale(month**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", taxi, 5, r_seed=3, target_log=True)

In [19]:
result_set

,R-square,AIC,BIC,Cond.No.,Pb(Fstatics),Pb(omnibus),Pb(jb),Dub-Wat,Remarks
0,0.631,1612414.563,1612630.018,50.908,0.0,0.0,0.0,1.998,log_duration ~ ...
1,0.631,1611578.819,1611794.275,50.955,0.0,0.0,0.0,2.000,log_duration ~ ...
2,0.631,1610687.359,1610902.815,50.943,0.0,0.0,0.0,1.999,log_duration ~ ...
3,0.632,1607032.039,1607247.495,50.933,0.0,0.0,0.0,1.999,log_duration ~ ...
4,0.631,1610471.057,1610686.512,50.960,0.0,0.0,0.0,2.000,log_duration ~ ...


In [20]:
score

[0.48104910830304554,
 0.4816145655639277,
 0.48229698387921804,
 0.4855740619608032,
 0.4824363062250503]

---

In [31]:
t2 = taxi.loc[:1000000]

In [24]:
# regularize
model = sm.OLS.from_formula("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*scale(work)\
                             + scale(sqrt_log_dist)*scale(month)\
                             + C(hour)\
                             + C(store_and_fwd_flag)\
                             + scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude) \
                             + 0", data = taxi)
result2 = model.fit_regularized(alpha=0.001, L1_wt=1)
print(result2.params)

C(vendor_id)[1]                           6.456048
C(vendor_id)[2]                           6.479196
C(hour)[T.1]                             -0.069420
C(hour)[T.2]                             -0.093841
C(hour)[T.3]                             -0.114388
C(hour)[T.4]                             -0.146976
C(hour)[T.5]                             -0.239676
C(hour)[T.6]                             -0.190974
C(hour)[T.7]                              0.000000
C(hour)[T.8]                              0.000000
C(hour)[T.9]                              0.004636
C(hour)[T.10]                             0.012672
C(hour)[T.11]                             0.048086
C(hour)[T.12]                             0.063467
C(hour)[T.13]                             0.056618
C(hour)[T.14]                             0.067970
C(hour)[T.15]                             0.072543
C(hour)[T.16]                             0.035323
C(hour)[T.17]                             0.033543
C(hour)[T.18]                  

In [13]:
model1 = sm.OLS.from_formula("log_duration ~ \
                               scale(sqrt_log_dist)*C(vendor_id)\
                             + scale(sqrt_log_dist)*scale(work)\
                             + scale(sqrt_log_dist)*scale(month)\
                             + C(hour)\
                             + C(store_and_fwd_flag)\
                             + scale(weather_event)\
                             + scale(weekday)+ scale(weekday**2)\
                             + scale(dropoff_latitude)+ scale(dropoff_longitude) \
                             + 0", data = taxi)
result = model1.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           log_duration   R-squared:                       0.719
Model:                            OLS   Adj. R-squared:                  0.719
Method:                 Least Squares   F-statistic:                 1.027e+05
Date:                Mon, 19 Mar 2018   Prob (F-statistic):               0.00
Time:                        23:46:23   Log-Likelihood:            -7.1439e+05
No. Observations:             1441360   AIC:                         1.429e+06
Df Residuals:                 1441323   BIC:                         1.429e+06
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
C(vendor_id)[1]                            6.3846      0.002   3446.527      0.000       6.381       6.388
C(vendor_id)[2]                            6.3836      0.002   3462.992      0.000       6.380       6.387
C(hour)[T.1]                              -0.0459      0.003    -17.156      0.000      -0.051      -0.041
C(hour)[T.2]                              -0.0804      0.003    -27.195      0.000      -0.086      -0.075
C(hour)[T.3]                              -0.1152      0.003    -35.204      0.000      -0.122      -0.109
C(hour)[T.4]                              -0.1714      0.004    -47.138      0.000      -0.178      -0.164
C(hour)[T.5]                              -0.2681      0.004    -72.341      0.000      -0.275      -0.261
C(hour)[T.6]                              -0.1712      0.003    -60.823      0.000      -0.177      -0.166
C(hour)[T.7]                               0.0680      0.002     27.683      0.000       0.063       0.073
C(hour)[T.8]                               0.0830      0.003     33.057      0.000       0.078       0.088
C(hour)[T.9]                               0.1301      0.002     52.310      0.000       0.125       0.135
C(hour)[T.10]                              0.1373      0.002     55.168      0.000       0.132       0.142
C(hour)[T.11]                              0.1711      0.002     69.511      0.000       0.166       0.176
C(hour)[T.12]                              0.1856      0.002     76.148      0.000       0.181       0.190
C(hour)[T.13]                              0.1764      0.002     72.337      0.000       0.172       0.181
C(hour)[T.14]                              0.1885      0.002     77.659      0.000       0.184       0.193
C(hour)[T.15]                              0.1936      0.002     79.338      0.000       0.189       0.198
C(hour)[T.16]                              0.1596      0.002     64.102      0.000       0.155       0.165
C(hour)[T.17]                              0.1535      0.002     63.582      0.000       0.149       0.158
C(hour)[T.18]                              0.1323      0.002     56.312      0.000       0.128       0.137
C(hour)[T.19]                              0.0516      0.002     21.939      0.000       0.047       0.056
C(hour)[T.20]                             -0.0389      0.002    -16.321      0.000      -0.044      -0.034
C(hour)[T.21]                              0.0817      0.002     36.682      0.000       0.077       0.086
C(hour)[T.22]                              0.0788      0.002     35.075      0.000       0.074       0.083
C(hour)[T.23]                              0.0383      0.002     16.586      0.000       0.034       0.043
C(store_and_fwd_flag)[T.1]                -0.0074      0.005     -1.651      0.099     

In [14]:
test = pd.read_csv("edited_test.csv")
test['sqrt_log_dist'] = test['dist'].apply(lambda x: np.sqrt(np.log1p(x)))

In [15]:
# 테스트 데이터를 통해 y값 예측
y_hat = result.predict(test)
y_hat = y_hat.apply(lambda x: (round(np.exp(x))))
ans = pd.concat([test['id'], y_hat], axis=1)
ans.rename(columns={'id':'id' , 0:'trip_duration'}, inplace=True)
ans.tail()

,id,trip_duration
625129,id3008929,303.0
625130,id3700764,1056.0
625131,id2568735,1210.0
625132,id1384355,1825.0
625133,id0621643,1055.0


In [16]:
ans[ans['trip_duration']>50000]

,id,trip_duration
32178,id1627730,5.050180e+05
324125,id0505885,1.052561e+31


In [18]:
test.loc[324125]

id                              id0505885
vendor_id                               2
pickup_datetime       2016-03-29 14:28:46
passenger_count                         2
pickup_longitude                 -121.933
pickup_latitude                   37.3896
dropoff_longitude                -121.933
dropoff_latitude                  37.3894
store_and_fwd_flag                      0
dist                            0.0295004
month                                   3
day                                    29
hour                                   14
weekday                                 1
holiday                                 0
work                                    1
weather_event                           1
sqrt_log_dist                     0.17051
Name: 324125, dtype: object

In [19]:
ans['trip_duration'].loc[324125] = 0

/home/mk/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
ans['trip_duration'].loc[324125]

0.0

In [21]:
ans['trip_duration'] = ans['trip_duration'].apply(lambda x: int(x))

In [22]:
# Kaggle 제출파일
ans.to_csv('basic_model.csv', index=False)

0.48294 - 757/1257


0.48361 - 758/1257 (60%)

In [16]:
757/1257

0.6022275258552108

---

## location

In [28]:
results = pd.DataFrame(columns = ["R-square", "AIC", "BIC", "Cond.No.", "Pb(Fstatics)", "Pb(omnibus)", "Pb(jb)", "Dub-Wat","Remarks"])

# origin data model
model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  s(location)')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(dropoff_latitude)+ scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o p_la')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o d_la')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o p_lo')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(dropoff_latitude)+ scale(pickup_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  w/o d_lo')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(pickup_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  pickup')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(dropoff_latitude)+ scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  dropoff')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_latitude) + scale(dropoff_latitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  lati')

model1 = sm.OLS.from_formula("log_duration ~ scale(sqrt_log_dist) + scale(pickup_longitude)+scale(dropoff_longitude)", data = taxi)
result1 = model1.fit()
storage(result1, results, 'sqrt dist +  logi')
results

,R-square,AIC,BIC,Cond.No.,Pb(Fstatics),Pb(omnibus),Pb(jb),Dub-Wat,Remarks
0,0.592,2159495.690,2159568.848,3.083,0.0,0.0,0.0,1.999,sqrt dist + s(location)
1,0.592,2159671.657,2159732.622,3.020,0.0,0.0,0.0,1.999,sqrt dist + w/o p_la
2,0.590,2166669.248,2166730.213,3.036,0.0,0.0,0.0,1.999,sqrt dist + w/o d_la
3,0.590,2166669.248,2166730.213,3.036,0.0,0.0,0.0,1.999,sqrt dist + w/o p_lo
4,0.592,2161066.150,2161127.115,1.786,0.0,0.0,0.0,1.999,sqrt dist + w/o d_lo
5,0.590,2168425.813,2168474.585,1.291,0.0,0.0,0.0,1.999,sqrt dist + pickup
6,0.592,2159880.439,2159929.211,1.248,0.0,0.0,0.0,1.999,sqrt dist + dropoff
7,0.592,2161680.626,2161729.397,1.779,0.0,0.0,0.0,1.999,sqrt dist + lati
8,0.590,2167634.814,2167683.586,2.993,0.0,0.0,0.0,1.999,sqrt dist + logi
